In [5]:
# Import modules
import pandas as pd
import numpy as np
from pathlib import Path
import sqlalchemy
%matplotlib inline
import hvplot.pandas

In [16]:
# Read in the csv file from the Resources folder
sp500_df = pd.read_csv(
    Path('./Resources/SPY.csv'),
    # index_col ='Date',
    parse_dates=True,
    infer_datetime_format=True
)

In [17]:
# Review the first 5 rows of sp500 returns
sp500_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-11-03,333.690002,338.250000,330.290009,336.029999,331.362213,93294200
1,2020-11-04,340.859985,347.940002,339.589996,343.540009,338.767914,126959700
2,2020-11-05,349.239990,352.190002,348.859985,350.239990,345.374847,82039700
3,2020-11-06,349.929993,351.510010,347.649994,350.160004,345.295959,74973000
4,2020-11-09,363.970001,364.380005,354.059998,354.559998,349.634827,172304200


In [18]:
# Create a SQLite database
database_connection_string ='sqlite:///'

In [19]:
# Create a database engine
engine = sqlalchemy.create_engine(database_connection_string,echo=True)
engine

Engine(sqlite:///)

In [20]:
# Read the table into a DataFrame
sp500_df.to_sql('sp500', engine, index=False, if_exists='replace')
sp500_df = pd.read_sql_table('sp500', con=engine)

2021-11-03 22:16:34,637 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sp500")
2021-11-03 22:16:34,639 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-03 22:16:34,641 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sp500")
2021-11-03 22:16:34,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-03 22:16:34,649 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-03 22:16:34,652 INFO sqlalchemy.engine.Engine 
CREATE TABLE sp500 (
	"Date" TEXT, 
	"Open" FLOAT, 
	"High" FLOAT, 
	"Low" FLOAT, 
	"Close" FLOAT, 
	"Adj Close" FLOAT, 
	"Volume" BIGINT
)


2021-11-03 22:16:34,654 INFO sqlalchemy.engine.Engine [no key 0.00237s] ()
2021-11-03 22:16:34,657 INFO sqlalchemy.engine.Engine COMMIT
2021-11-03 22:16:34,661 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-03 22:16:34,679 INFO sqlalchemy.engine.Engine INSERT INTO sp500 ("Date", "Open", "High", "Low", "Close", "Adj Close", "Volume") VALUES (?, ?, ?, ?, ?, ?, ?)
2021-11-03 22:16:34,680 INFO sqlalchemy.engine.Engine 

In [21]:
# Run the database engine to see that the sp500 table was created in the database
engine.table_names()

2021-11-03 22:16:38,681 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-11-03 22:16:38,682 INFO sqlalchemy.engine.Engine [raw sql] ()


/Users/patrickbeeson/opt/anaconda3/envs/sqlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['sp500']

In [22]:
# Create a query for all the rows and columns in the sp500 table
sp500_query = f"""
SELECT *
FROM sp500
"""
# Display the results of the sp500 query in a new DataFrame
sp500_results_df=pd.read_sql(sp500_query, engine, parse_dates='Date', index_col='Date')
sp500_results_df

2021-11-03 22:16:41,074 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
SELECT *
FROM sp500
")
2021-11-03 22:16:41,076 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-03 22:16:41,078 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
SELECT *
FROM sp500
")
2021-11-03 22:16:41,080 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-03 22:16:41,082 INFO sqlalchemy.engine.Engine 
SELECT *
FROM sp500

2021-11-03 22:16:41,084 INFO sqlalchemy.engine.Engine [raw sql] ()


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-11-03,333.690002,338.250000,330.290009,336.029999,331.362213,93294200
2020-11-04,340.859985,347.940002,339.589996,343.540009,338.767914,126959700
2020-11-05,349.239990,352.190002,348.859985,350.239990,345.374847,82039700
2020-11-06,349.929993,351.510010,347.649994,350.160004,345.295959,74973000
2020-11-09,363.970001,364.380005,354.059998,354.559998,349.634827,172304200
...,...,...,...,...,...,...
2021-10-27,456.450012,457.160004,453.859985,453.940002,453.940002,72438000
2021-10-28,455.459991,458.399994,455.450012,458.320007,458.320007,51437900
2021-10-29,455.869995,459.559998,455.559998,459.250000,459.250000,70108200


In [23]:
# Drop the Open, High, Low, Close and Volume Columns, keep only Date and Volume.
sp500_results_df = sp500_results_df.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])

In [24]:
sp500_results_df.dtypes

Adj Close    float64
dtype: object

# Calculate the Daily Returns 1, 3, 5, 10 year

In [25]:
# Calculate the daily return of the sp500 over the last 10 years
sp500_results_df['daily_return'] = (sp500_results_df['Adj Close']/ sp500_results_df['Adj Close'].shift(1))-1

sp500_results_df


,Adj Close,daily_return
Date,,
2020-11-03,331.362213,NaN
2020-11-04,338.767914,0.022349
2020-11-05,345.374847,0.019503
2020-11-06,345.295959,-0.000228
2020-11-09,349.634827,0.012566
...,...,...
2021-10-27,453.940002,-0.004430
2021-10-28,458.320007,0.009649
2021-10-29,459.250000,0.002029


In [26]:
# Calculate 1 year daily returns using loc
daily_returns_1_year = sp500_results_df.loc['2020-11-01':'2021-11-01']
                                                           
daily_returns_1_year

,Adj Close,daily_return
Date,,
2020-11-03,331.362213,NaN
2020-11-04,338.767914,0.022349
2020-11-05,345.374847,0.019503
2020-11-06,345.295959,-0.000228
2020-11-09,349.634827,0.012566
...,...,...
2021-10-26,455.959991,0.000900
2021-10-27,453.940002,-0.004430
2021-10-28,458.320007,0.009649


In [27]:
# Calculate 3 year daily returns using loc
daily_returns_3_year = sp500_results_df.loc['2018-11-01':'2021-11-01']
                                                           
daily_returns_3_year

,Adj Close,daily_return
Date,,
2020-11-03,331.362213,NaN
2020-11-04,338.767914,0.022349
2020-11-05,345.374847,0.019503
2020-11-06,345.295959,-0.000228
2020-11-09,349.634827,0.012566
...,...,...
2021-10-26,455.959991,0.000900
2021-10-27,453.940002,-0.004430
2021-10-28,458.320007,0.009649


In [28]:

# Calculate 5 year daily returns using loc
daily_returns_5_year = sp500_results_df.loc['2016-11-01':'2021-11-01']
                                                           
daily_returns_5_year 

,Adj Close,daily_return
Date,,
2020-11-03,331.362213,NaN
2020-11-04,338.767914,0.022349
2020-11-05,345.374847,0.019503
2020-11-06,345.295959,-0.000228
2020-11-09,349.634827,0.012566
...,...,...
2021-10-26,455.959991,0.000900
2021-10-27,453.940002,-0.004430
2021-10-28,458.320007,0.009649


In [29]:
# Calculate 10 year daily returns using loc
daily_returns_10_year = sp500_results_df.loc['2011-11-01':'2021-11-01']
                                                           
daily_returns_10_year 

,Adj Close,daily_return
Date,,
2020-11-03,331.362213,NaN
2020-11-04,338.767914,0.022349
2020-11-05,345.374847,0.019503
2020-11-06,345.295959,-0.000228
2020-11-09,349.634827,0.012566
...,...,...
2021-10-26,455.959991,0.000900
2021-10-27,453.940002,-0.004430
2021-10-28,458.320007,0.009649


# Calculate the cummulative returns 1,3, 5, 10 Year

In [30]:
# Calculate the 1 year cumulative returns using the cumprod() function

sp500_cumulative_1_year_return = (1 + daily_returns_1_year['daily_return']).cumprod()-1
sp500_cumulative_1_year_return

Date
2020-11-03         NaN
2020-11-04    0.022349
2020-11-05    0.042288
2020-11-06    0.042050
2020-11-09    0.055144
                ...   
2021-10-26    0.376017
2021-10-27    0.369921
2021-10-28    0.383139
2021-10-29    0.385946
2021-11-01    0.388330
Name: daily_return, Length: 251, dtype: float64

In [31]:
# Calculate the 3 year cumulative returns using the cumprod() function

sp500_cumulative_3_year_return = (1 + daily_returns_3_year['daily_return']).cumprod()-1
sp500_cumulative_3_year_return

Date
2020-11-03         NaN
2020-11-04    0.022349
2020-11-05    0.042288
2020-11-06    0.042050
2020-11-09    0.055144
                ...   
2021-10-26    0.376017
2021-10-27    0.369921
2021-10-28    0.383139
2021-10-29    0.385946
2021-11-01    0.388330
Name: daily_return, Length: 251, dtype: float64

In [32]:
# Calculate the 5 year cumulative returns using the cumprod() function

sp500_cumulative_5_year_return = (1 + daily_returns_5_year['daily_return']).cumprod()-1

sp500_cumulative_5_year_return

Date
2020-11-03         NaN
2020-11-04    0.022349
2020-11-05    0.042288
2020-11-06    0.042050
2020-11-09    0.055144
                ...   
2021-10-26    0.376017
2021-10-27    0.369921
2021-10-28    0.383139
2021-10-29    0.385946
2021-11-01    0.388330
Name: daily_return, Length: 251, dtype: float64

In [33]:
# Calculate the 10 year cumulative returns using the cumprod() function

sp500_cumulative_10_year_return = (1 + daily_returns_10_year['daily_return']).cumprod()-1

sp500_cumulative_10_year_return

Date
2020-11-03         NaN
2020-11-04    0.022349
2020-11-05    0.042288
2020-11-06    0.042050
2020-11-09    0.055144
                ...   
2021-10-26    0.376017
2021-10-27    0.369921
2021-10-28    0.383139
2021-10-29    0.385946
2021-11-01    0.388330
Name: daily_return, Length: 251, dtype: float64

# Calculate the 1, 3, 5, 10 Year Mean 

In [34]:
# Calculate the sp500 1 year mean
SP500_1_year_mean = daily_returns_1_year['daily_return'].mean()

SP500_1_year_mean


0.0013445017279040238

In [35]:
# Calculate the sp500 3 year mean
SP500_3_year_mean = daily_returns_3_year['daily_return'].mean()
SP500_3_year_mean

0.0013445017279040238

In [36]:
# Calculate the sp500 5 year mean
SP500_5_year_mean = daily_returns_5_year['daily_return'].mean()
SP500_5_year_mean

0.0013445017279040238

In [37]:
# Calculate the sp500 10 year mean
SP500_10_year_mean = daily_returns_10_year['daily_return'].mean()
SP500_10_year_mean

0.0013445017279040238

# Calculate the standard deviation for each period

In [38]:
# One year standard deviation
sp500_std_dev_1_year = daily_returns_1_year['daily_return'].std()
sp500_std_dev_1_year

0.007919389889299982

In [39]:
# Three year standard deviation
sp500_std_dev_3_year = daily_returns_3_year['daily_return'].std()
sp500_std_dev_3_year

0.007919389889299982

In [40]:
# Five year standard deviation
sp500_std_dev_5_year = daily_returns_5_year['daily_return'].std()
sp500_std_dev_5_year

0.007919389889299982

In [41]:
# Ten year standard deviation
sp500_std_dev_10_year = daily_returns_10_year['daily_return'].std()
sp500_std_dev_10_year

0.007919389889299982

# Calculate the Annualized Returns

In [42]:
# SP500 one year annualized return
# set variable Trading Days to 252 days per year the market is open

trading_days = 252

sp500_1_year_annualized_return = (daily_returns_1_year['daily_return'].mean() * trading_days) *100

sp500_1_year_annualized_return

33.8814435431814

In [43]:
# SP500 3 year annualized return

sp500_3_year_annualized_return = (daily_returns_3_year['daily_return'].mean() * trading_days) *100
sp500_3_year_annualized_return

33.8814435431814

In [44]:
# SP500 5 year annualized return

sp500_5_year_annualized_return = (daily_returns_5_year['daily_return'].mean() * trading_days) * 100
sp500_5_year_annualized_return

33.8814435431814

In [45]:
# SP500 10 year annualized return

sp500_10_year_annualized_return = (daily_returns_10_year['daily_return'].mean() * trading_days) * 100
sp500_10_year_annualized_return

33.8814435431814

# Summary Stats for each time period (1, 3, 5, 10)

In [46]:
# Daily Returns 1 year - summary stats
daily_returns_1_year.describe()

,Adj Close,daily_return
count,251.000000,250.000000
mean,406.701965,0.001345
std,31.949118,0.007919
min,331.362213,-0.024441
25%,378.898529,-0.002809
50%,413.388855,0.001520
75%,434.768447,0.006167
max,460.040009,0.024240


In [47]:
# Daily Returns 3 year - summary stats
daily_returns_3_year.describe()

,Adj Close,daily_return
count,251.000000,250.000000
mean,406.701965,0.001345
std,31.949118,0.007919
min,331.362213,-0.024441
25%,378.898529,-0.002809
50%,413.388855,0.001520
75%,434.768447,0.006167
max,460.040009,0.024240


In [48]:
# Daily Returns 5 year - summary stats
daily_returns_5_year.describe()

,Adj Close,daily_return
count,251.000000,250.000000
mean,406.701965,0.001345
std,31.949118,0.007919
min,331.362213,-0.024441
25%,378.898529,-0.002809
50%,413.388855,0.001520
75%,434.768447,0.006167
max,460.040009,0.024240


In [49]:
# Daily Returns 10 year - summary stats
daily_returns_10_year.describe()

,Adj Close,daily_return
count,251.000000,250.000000
mean,406.701965,0.001345
std,31.949118,0.007919
min,331.362213,-0.024441
25%,378.898529,-0.002809
50%,413.388855,0.001520
75%,434.768447,0.006167
max,460.040009,0.024240


# Annualized Standard Deviation

In [50]:
# Calculate the Annual Standard Deviation on ten years of SP500 daily returns

annual_std_dev_sp500_10_year = daily_returns_10_year.std() * np.sqrt(trading_days)

# Display the results
annual_std_dev_sp500_10_year

Adj Close       507.176521
daily_return      0.125716
dtype: float64

In [51]:
# Calculate the Sharpe Ratio on the 10 year sp500 return
sharpe_ratio_sp500_10_year = sp500_10_year_annualized_return / annual_std_dev_sp500_10_year
sharpe_ratio_sp500_10_year

Adj Close         0.066804
daily_return    269.506913
dtype: float64